In [11]:
import pandas as pd
import copy
import warnings
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report,confusion_matrix

from sklearn.neural_network import MLPClassifier

from sklearn.model_selection import train_test_split

In [12]:
# Read input file

input_file = pd.read_csv('./Final_Files/sample_refined_updated.csv')

# Shuffle data frame

input_file = input_file.sample(frac=1)
input_file.head()

,acousticness,analysis_url,danceability,duration_ms,energy,track_id,instrumentalness,key,liveness,loudness,...,track_href,type,uri,valence,artist_name,track_name,popularity,bbhot,artist_popularity,artist_followers
2360,0.07750,https://api.spotify.com/v1/audio-analysis/3uPu...,0.720,155653,0.628,3uPuEFKEzGTn628CCQZUCF,0.000005,5,0.0976,-7.192,...,https://api.spotify.com/v1/tracks/3uPuEFKEzGTn...,audio_features,spotify:track:3uPuEFKEzGTn628CCQZUCF,0.5910,6ix9ine,DUMMY (feat. TrifeDrew),67,0,88,3886564
2393,0.08450,https://api.spotify.com/v1/audio-analysis/154J...,0.487,266766,0.630,154J7EMKDUOVwD2hWGDNHU,0.000000,8,0.1090,-6.053,...,https://api.spotify.com/v1/tracks/154J7EMKDUOV...,audio_features,spotify:track:154J7EMKDUOVwD2hWGDNHU,0.1560,Dermot Kennedy,Young & Free,65,0,76,247030
1167,0.00584,https://api.spotify.com/v1/audio-analysis/5HR1...,0.558,220606,0.493,5HR1TFot7cGc7TmLOjd49N,0.277000,10,0.1100,-10.631,...,https://api.spotify.com/v1/tracks/5HR1TFot7cGc...,audio_features,spotify:track:5HR1TFot7cGc7TmLOjd49N,0.0666,Danger Incorporated,Forever,56,1,60,53924
3464,0.34800,https://api.spotify.com/v1/audio-analysis/5Pxb...,0.781,79205,0.335,5Pxb3DpymMiRl8JDiOXTPE,0.916000,10,0.1900,-9.921,...,https://api.spotify.com/v1/tracks/5Pxb3DpymMiR...,audio_features,spotify:track:5Pxb3DpymMiRl8JDiOXTPE,0.5520,Lost Son,Life,60,1,53,2104
1186,0.03350,https://api.spotify.com/v1/audio-analysis/1Szf...,0.781,182723,0.487,1Szfm2XAx3Kgl55dyl8sTU,0.000000,8,0.1630,-8.823,...,https://api.spotify.com/v1/tracks/1Szfm2XAx3Kg...,audio_features,spotify:track:1Szfm2XAx3Kgl55dyl8sTU,0.3440,Lil Baby,Time (feat. Meek Mill),72,0,89,2057232


In [13]:
## Dropping fields (strings/objects)

'''
drop_list = ['analysis_url','track_id', 'track_href', 'type', 'uri', 
             'artist_name', 'track_name']
'''
drop_list = ['analysis_url','track_id', 'track_href', 'type', 'uri', 
             'artist_name', 'track_name', 'time_signature','popularity']

train = input_file.drop(drop_list, axis = 1)

columns = ['acousticness', 'danceability', 'duration_ms', 'energy', 'instrumentalness',
           'liveness', 'loudness', 'speechiness', 'tempo', 'mode', 'key', 'valence', 'artist_popularity','artist_followers']

train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 2360 to 4944
Data columns (total 15 columns):
acousticness         5000 non-null float64
danceability         5000 non-null float64
duration_ms          5000 non-null int64
energy               5000 non-null float64
instrumentalness     5000 non-null float64
key                  5000 non-null int64
liveness             5000 non-null float64
loudness             5000 non-null float64
mode                 5000 non-null int64
speechiness          5000 non-null float64
tempo                5000 non-null float64
valence              5000 non-null float64
bbhot                5000 non-null int64
artist_popularity    5000 non-null int64
artist_followers     5000 non-null int64
dtypes: float64(9), int64(6)
memory usage: 625.0 KB


In [14]:
## Making a copy of dataframe target variable and assigning it to Y
Y = copy.deepcopy(train.bbhot)
Y.shape

(5000,)

In [15]:
## Dropping the target attribute and making new dataframe

train_1 = train.drop("bbhot", axis=1)
train_1.head()

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,valence,artist_popularity,artist_followers
2360,0.07750,0.720,155653,0.628,0.000005,5,0.0976,-7.192,0,0.0718,90.985,0.5910,88,3886564
2393,0.08450,0.487,266766,0.630,0.000000,8,0.1090,-6.053,1,0.0432,153.131,0.1560,76,247030
1167,0.00584,0.558,220606,0.493,0.277000,10,0.1100,-10.631,0,0.0491,105.056,0.0666,60,53924
3464,0.34800,0.781,79205,0.335,0.916000,10,0.1900,-9.921,0,0.2020,87.987,0.5520,53,2104
1186,0.03350,0.781,182723,0.487,0.000000,8,0.1630,-8.823,1,0.5890,95.879,0.3440,89,2057232


In [16]:
train_1.describe()

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,valence,artist_popularity,artist_followers
count,5000.000000,5000.000000,5.000000e+03,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.00000,5000.000000,5000.000000,5000.000000,5000.000000,5.000000e+03
mean,0.266222,0.650610,2.038833e+05,0.593527,0.065873,5.275800,0.171321,-7.430599,0.58780,0.124946,120.346522,0.436074,73.737000,3.206522e+06
std,0.284436,0.161101,5.151627e+04,0.203449,0.216466,3.618612,0.126541,4.247411,0.49228,0.118186,30.120835,0.219097,14.919972,5.972459e+06
min,0.000003,0.000000,3.524000e+04,0.000900,0.000000,0.000000,0.019600,-41.537000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000e+00
25%,0.038775,0.552000,1.771382e+05,0.474000,0.000000,2.000000,0.098575,-8.463250,0.00000,0.041400,95.980000,0.268000,65.000000,1.389320e+05
50%,0.153000,0.667000,2.017070e+05,0.609000,0.000000,5.000000,0.120000,-6.443000,1.00000,0.069400,119.975500,0.422000,75.000000,7.818130e+05
75%,0.415000,0.767000,2.281582e+05,0.741250,0.000182,8.000000,0.199000,-5.010500,1.00000,0.175250,141.987000,0.592000,85.000000,3.138132e+06
max,0.996000,0.972000,1.355938e+06,0.998000,1.000000,11.000000,0.979000,0.605000,1.00000,0.966000,220.099000,0.990000,100.000000,4.368682e+07


In [17]:
## Creating standard scaler object

warnings.filterwarnings("ignore")

#scaler = StandardScaler()

scaler = MinMaxScaler()

## Fit scaler to features

scaler.fit(train_1)

# Transform features to scaled version

scaled_features = scaler.transform(train_1)

#Convert the scaled features to a dataframe and check the head of this dataframe to make sure the scaling worked.

#df_feat = pd.DataFrame(scaled_features,columns=train.columns[:-1])

df_feat = pd.DataFrame(scaled_features,columns=train_1.columns)

df_feat.head()

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,valence,artist_popularity,artist_followers
0,0.077808,0.740741,0.091174,0.628924,0.000005,0.454545,0.081301,0.814983,0.0,0.074327,0.413382,0.596970,0.88,0.088964
1,0.084836,0.501029,0.175306,0.630930,0.000000,0.727273,0.093183,0.842010,1.0,0.044720,0.695737,0.157576,0.76,0.005655
2,0.005860,0.574074,0.140355,0.493531,0.277000,0.909091,0.094226,0.733378,0.0,0.050828,0.477312,0.067273,0.60,0.001234
3,0.349395,0.803498,0.033289,0.335072,0.916000,0.909091,0.177611,0.750225,0.0,0.209110,0.399761,0.557576,0.53,0.000048
4,0.033631,0.803498,0.111670,0.487514,0.000000,0.727273,0.149468,0.776280,1.0,0.609731,0.435618,0.347475,0.89,0.047090


In [18]:
## NN Model

X = df_feat
y = Y

X_train, X_test, y_train, y_test = train_test_split(df_feat, Y, test_size=0.30, random_state=7)
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)
clf.fit(X_train, y_train)



MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(5, 2), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=1, shuffle=True, solver='lbfgs', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [19]:
predictions = clf.predict(X)


print("Prediction Accuracy Without Cross Validation: %.2f%%" % (clf.score(X, y) * 100))

print("Confusion Matrix: ")
print(confusion_matrix(y,predictions))


print("Classification Report: ")
print(classification_report(y,predictions))

Prediction Accuracy Without Cross Validation: 60.32%
Confusion Matrix: 
[[1589  911]
 [1073 1427]]
Classification Report: 
              precision    recall  f1-score   support

           0       0.60      0.64      0.62      2500
           1       0.61      0.57      0.59      2500

   micro avg       0.60      0.60      0.60      5000
   macro avg       0.60      0.60      0.60      5000
weighted avg       0.60      0.60      0.60      5000



In [20]:
#Function to evaluate my model with Cross validation

from sklearn.model_selection import cross_val_score

def testingModel(model, X_train, Y_train):
    scores = cross_val_score(model, X_train, Y_train, cv=10, scoring = "roc_auc")
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard Deviation:", scores.std())
    return scores.mean()

In [21]:
## Implementing NN with cross validation

mlp_c = MLPClassifier()
mlp_c.fit(df_feat, Y)

acc_nn = testingModel(mlp_c, df_feat, Y)
print("Prediction Accuracy With 10-Fold Cross Validation(Mean Value): %.2f%%" % (acc_nn * 100.0))

Scores: [0.699952 0.673104 0.67072  0.618768 0.672256 0.6604   0.651664 0.634528
 0.652608 0.621936]
Mean: 0.6555936
Standard Deviation: 0.024001876406647873
Prediction Accuracy With 10-Fold Cross Validation(Mean Value): 65.56%


In [14]:
## Implementing LDA (Linear Discriminant analysis) with cross validation

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda_c = LinearDiscriminantAnalysis()

lda_c.fit(df_feat, Y)

acc_lda = testingModel(lda_c, df_feat, Y)
print("Accuracy: %.2f%%" % (acc_nn * 100.0))

Scores: [0.618352 0.669456 0.61928  0.627872 0.649952 0.62744  0.61608  0.6248
 0.584752 0.628448]
Mean: 0.6266432
Standard Deviation: 0.020915186046506988
Accuracy: 64.44%


In [22]:
print("List of Hyperparameters: ")
print("Activation Function:",clf.activation, "|Solver:", clf.solver)

List of Hyperparameters: 
Activation Function: relu |Solver: lbfgs
